<h2>0. Data Generation</h2>
<h4>Genrate Synthetic Patient Data through Synthea</h4>
1. Go to link: <a>https://github.com/synthetichealth/synthea</a>, follow Developer Quick Start and install Synthea. </p>
2. Go to <span style="color:powderblue;">.../synthea/src/main/resources/synthea.properties</span>, change <span style="color:powderblue;">exporter.csv.export = true</span>. </p>
3. Open cmd, input <span style="color:powderblue;">cd .../synthea</span>, then <span style="color:powderblue;">run_synthea -s 1256 -p 1000 California "Los Angeles"</span>. This generate 1000 patients' information located at California, Los Angeles with seed 1256.</p>
4. In .../synthea/output/csv, locate following files: patients.csv, medications.csv, conditions.csv.</p>




<h2>1. Data Localization</h2>
<h4>Change US info to Australia</h4>

Given that Synthea defaultly generates US address, Consider manually change the following columns for better data visualization: </p>
 1. ETHNICITY: From "hispanic" and "non-hispanic" to "Aboriginal and Torres Strait Islander" and "other". Change ratio of Aboriginal and Torres Strait Islander to 3.2% (based on the 2021 census). </p>
 2. CITY, STATE，COUNTY，ZIP，LAT，LON：The geomap mainly focues on Victoria region. Therefore STATE are all changed to Victoria, and COUNTY are randomly assign to any Victoria suburb. CITY, ZIP, LAT, LON asign according to the Database of Australian Postcodes: <a>https://www.matthewproctor.com/australian_postcodes</a>.Suburb name is based on VIC Local Government Areas GDA2020 (2nd intake) : <a>https://data.gov.au/dataset/ds-dga-bdf92691-c6fe-42b9-a0e2-a4cd716fa811/details</a></p>

 Reason why we leave BIRTHPLACE, ADDRESS，FIPS unchanged:</p>
Birthplace and address is highly variated. They are not likely to play a role in data visualization. FIPS is a unique geography identifier used by the US. This system does not apply to Austalia. Therefore, these are leaved unchanged.

In [1]:
import pandas as pd
import random
import geopandas as gpd
random.seed(1256)

#Import .shp file for suburb info
geo_state = gpd.read_file('data/sf/vic_localities.shp')

#Import genetrated Synthea data
patients_df = pd.read_csv('data/Synthetic data/patients.csv')

1. ETHNICITY

In [2]:
#Generate ethnicities with the proportions
patients_df['ETHNICITY'] = random.choices(
    ["Other", "Aboriginal and Torres Strait Islander"],
    weights=[0.968, 0.032],  
    k=len(patients_df)
)

patients_df.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,f5502975-21b1-6a26-00ba-0479ee7013f1,2024-01-01,NaN,999-22-4042,NaN,NaN,NaN,Dorian295,Predovic534,NaN,...,Los Angeles,California,Los Angeles County,6037,90095,33.837180,-118.255232,1176.77,0.00,142607
1,42bf13d8-30b7-23dd-0bab-338a9e445071,2023-05-31,NaN,999-20-8605,NaN,NaN,NaN,Neoma676,Hoppe518,NaN,...,Los Angeles,California,Los Angeles County,6037,91403,34.238998,-118.452832,3165.73,0.00,145231
2,8842ad59-b185-c6ce-871a-dcdcf111757a,2013-10-20,NaN,999-67-4381,NaN,NaN,NaN,Enrique929,Vigil207,NaN,...,Los Angeles,California,Los Angeles County,6037,91405,34.310508,-118.570361,1350.00,18394.78,29236
3,d0c2d7e5-2e4a-5d6a-cd68-70824b671847,1983-03-19,NaN,999-10-2867,S99914477,X56703990X,Mr.,Teodoro374,Quintana711,NaN,...,Los Angeles,California,Los Angeles County,6037,90031,34.207267,-118.333245,53004.30,1113942.73,388848
4,4d7dbb8a-391d-6298-c8ad-ae73dba63cc1,1985-03-13,NaN,999-34-7414,S99956571,X6519174X,Mr.,Marcelino726,Schulist381,NaN,...,Los Angeles,California,Los Angeles County,6037,90710,33.980496,-118.334913,58515.31,1099710.08,44926


 2. CITY, STATE，COUNTY，ZIP，LAT，LON：

In [3]:
#Change STATE to Victoria
patients_df['STATE'] = "Victoria"

#Get suburb name from .shp file and drop None
Victoria_suburb = geo_state['LOC_NAME'].tolist()

#Randomly assign Australia suburb to original dataframe 
patients_df["COUNTY"] = random.choices(Victoria_suburb, k=len(patients_df))
patients_df.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,f5502975-21b1-6a26-00ba-0479ee7013f1,2024-01-01,NaN,999-22-4042,NaN,NaN,NaN,Dorian295,Predovic534,NaN,...,Los Angeles,Victoria,Keilor East,6037,90095,33.837180,-118.255232,1176.77,0.00,142607
1,42bf13d8-30b7-23dd-0bab-338a9e445071,2023-05-31,NaN,999-20-8605,NaN,NaN,NaN,Neoma676,Hoppe518,NaN,...,Los Angeles,Victoria,Old Tallangatta,6037,91403,34.238998,-118.452832,3165.73,0.00,145231
2,8842ad59-b185-c6ce-871a-dcdcf111757a,2013-10-20,NaN,999-67-4381,NaN,NaN,NaN,Enrique929,Vigil207,NaN,...,Los Angeles,Victoria,Tyrrell,6037,91405,34.310508,-118.570361,1350.00,18394.78,29236
3,d0c2d7e5-2e4a-5d6a-cd68-70824b671847,1983-03-19,NaN,999-10-2867,S99914477,X56703990X,Mr.,Teodoro374,Quintana711,NaN,...,Los Angeles,Victoria,Myall,6037,90031,34.207267,-118.333245,53004.30,1113942.73,388848
4,4d7dbb8a-391d-6298-c8ad-ae73dba63cc1,1985-03-13,NaN,999-34-7414,S99956571,X6519174X,Mr.,Marcelino726,Schulist381,NaN,...,Los Angeles,Victoria,Buchan South,6037,90710,33.980496,-118.334913,58515.31,1099710.08,44926


To modify the ZIP, LAT, LON, we can refer to the Database of Australian Postcodes: <a>https://www.matthewproctor.com/australian_postcodes</a>

In [4]:
#Modify CITY, ZIP, LAT, LON accordingly
postcodes_df = pd.read_csv("data/australian_postcodes.csv")

# Map COUNTY to locality in postcodes_df to get corresponding ZIP, LAT, LON
county_to_zip_lat_lon = dict(zip(postcodes_df["locality"].str.title(), 
                                 postcodes_df[["postcode", "lat", "long", "lgaregion"]].values.tolist()))
county_to_zip_lat_lon

{'Anu': [200, -35.2777, 149.119, 'Unincorporated ACT'],
 'Australian National University': [200,
  -35.2777,
  149.1189,
  'Unincorporated ACT'],
 'Darwin': [801, -12.458684, 130.83668, 'Darwin Waterfront Precinct'],
 'Darwin City': [800, -12.458684, 130.83668, 'Darwin Waterfront Precinct'],
 'Wagait Beach': [822, -11.75841975, 130.6254299, 'Tiwi Islands'],
 'Parap': [820, -12.428017, 130.873315, 'Darwin'],
 'Alawa': [810, -12.381806, 130.866242, 'Darwin'],
 'Brinkin': [810, -12.381806, 130.866242, 'Darwin'],
 'Casuarina': [6167, -32.223657, 115.835308, 'Kwinana'],
 'Coconut Grove': [810, -12.39614166, 130.8502369, 'Darwin'],
 'Jingili': [810, -12.381806, 130.866242, 'Darwin'],
 'Lee Point': [810, -12.381806, 130.866242, 'Darwin'],
 'Lyons': [4124, -27.701681, 152.967761, 'Logan'],
 'Millner': [810, -12.381806, 130.866242, 'Darwin'],
 'Moil': [810, -12.381806, 130.866242, 'Darwin'],
 'Muirhead': [810, -12.35811947, 130.8959029, 'Darwin'],
 'Nakara': [810, -12.381806, 130.866242, 'Darwi

In [5]:
# Ensure the mapping returns a tuple of (ZIP, LAT, LON, CITY) for every row, even if some mappings are None
patients_df['mapped'] = patients_df['COUNTY'].map(county_to_zip_lat_lon).apply(lambda x: x if isinstance(x, list) else [None, None, None, None])

# Now, separate this mapped data into individual columns
patients_df['ZIP'] = patients_df['mapped'].apply(lambda x: x[0] if x[0] is not None else 0).astype(int)
patients_df['LAT'] = patients_df['mapped'].apply(lambda x: x[1] if x[1] is not None else 0)
patients_df['LON'] = patients_df['mapped'].apply(lambda x: x[2] if x[2] is not None else 0)
patients_df['CITY'] = patients_df['mapped'].apply(lambda x: x[3] if x[3] is not None else 'Unknown')

# Drop the temporary 'mapped' column as it's no longer needed
patients_df.drop('mapped', axis=1, inplace=True)

patients_df.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,f5502975-21b1-6a26-00ba-0479ee7013f1,2024-01-01,NaN,999-22-4042,NaN,NaN,NaN,Dorian295,Predovic534,NaN,...,Moonee Valley,Victoria,Keilor East,6037,3033,-37.741070,144.857638,1176.77,0.00,142607
1,42bf13d8-30b7-23dd-0bab-338a9e445071,2023-05-31,NaN,999-20-8605,NaN,NaN,NaN,Neoma676,Hoppe518,NaN,...,Wodonga,Victoria,Old Tallangatta,6037,3701,-36.201939,147.249954,3165.73,0.00,145231
2,8842ad59-b185-c6ce-871a-dcdcf111757a,2013-10-20,NaN,999-67-4381,NaN,NaN,NaN,Enrique929,Vigil207,NaN,...,Swan Hill,Victoria,Tyrrell,6037,3533,-35.384557,142.839304,1350.00,18394.78,29236
3,d0c2d7e5-2e4a-5d6a-cd68-70824b671847,1983-03-19,NaN,999-10-2867,S99914477,X56703990X,Mr.,Teodoro374,Quintana711,NaN,...,Gannawarra,Victoria,Myall,6037,3579,-35.767974,143.769445,53004.30,1113942.73,388848
4,4d7dbb8a-391d-6298-c8ad-ae73dba63cc1,1985-03-13,NaN,999-34-7414,S99956571,X6519174X,Mr.,Marcelino726,Schulist381,NaN,...,East Gippsland,Victoria,Buchan South,6037,3885,-37.291807,148.118511,58515.31,1099710.08,44926


In [6]:
#explore patients.csv
patients_df.to_csv("data/modified data/patients.csv", index=False)

Confrim that the suburb name matches the shape file.

In [7]:
all_elements_present = patients_df['COUNTY'].isin(geo_state['LOC_NAME']).all()
print(all_elements_present)

True
